In [41]:


import os, sys, subprocess, time, toml
import pandas as pd
from MOASMO_parameter_allbasin_emulator import allbasin_emulator_train_and_optimize
import run_multiple_paramsets_Derecho


########################################################################################################################
# build emulator and generate outputs

iter_end = 1 # e.g., iter_end=2 means outputs from iter0 and iter1 will be used to generate new paprameters for iter 2
ncpus = 1
# ncpus = 20
# iterend = 1
numruns = 100

infile_basin_info = f"/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv"
infile_param_info = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/src/parameter/CTSM_CAMELS_SA_param_240202.csv'
infile_attr_foruse = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/data/camels_attributes_table_TrainModel.csv'
inpath_moasmo = "/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange"
path_CTSM_case = f'/glade/work/guoqiang/CTSM_CAMELS/Calib_HH_MOASMO_bigrange'

In [43]:


for tarbasin in range(627):
    config_file = f'{path_CTSM_case}/configuration/_level1-{tarbasin}_config_MOASMO.toml'
    config = toml.load(config_file)
    
    # inputs
    file_parameter_list = config['file_calib_param']
    path_CTSM_base = config['path_CTSM_case']
    path_script_MOASMO = config['path_script_MOASMO']
    path_CTSM_source = config['path_CTSM_source']
    ref_streamflow = config['file_Qobs']
    
    if 'add_flow_file' in config:
        add_flow_file = config['add_flow_file']
    else:
        add_flow_file = 'NA'
    
    script_singlerun = f'{path_script_MOASMO}/run_one_paramset_Derecho.py'
    script_clone = f'{path_CTSM_source}/cime/scripts/create_clone'

    if config['path_calib'] == 'NA':
        path_MOASMOcalib = f'{path_CTSM_base}_MOASMOcalib'
    else:
        path_MOASMOcalib = config['path_calib']
        
    # outputs
    path_paramset = f'{path_MOASMOcalib}/param_sets_emutest'
    path_submit = f'{path_MOASMOcalib}/run_model_emutest'
    path_archive = f'{path_MOASMOcalib}/ctsm_outputs_emutest'
        
    os.makedirs(path_MOASMOcalib, exist_ok=True) 
    
    # evaluation period
    RUN_STARTDATE = config['RUN_STARTDATE']
    ignore_month = config['ignore_month']
    STOP_OPTION = config['STOP_OPTION']
    STOP_N = config['STOP_N']
    
    if 'nonstandard_evaluation' in config:
        nonstandard_evaluation = config['nonstandard_evaluation']
    else:
        nonstandard_evaluation = 'NA'
    
    # HPC job settings
    job_mode = config['job_mode']
    job_CTSMiteration = config['job_CTSMiteration']
    # job_controlMOASMO = config['job_controlMOASMO'] # not needed here
    
    date_start = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(months=ignore_month)).strftime('%Y-%m-%d') # ignor the first year when evaluating model
    if STOP_OPTION == 'nyears':
        date_end = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(years=STOP_N)).strftime('%Y-%m-%d')
    elif STOP_OPTION == 'nmonths':
        date_end = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(months=STOP_N)).strftime('%Y-%m-%d')
    else:
        sys.exit(f'STOP_OPTION must be nyears or nmonths. {STOP_OPTION} is not accepted.')

    
    # generate submission commands (note, this won't submit a real job on Derecho)
    run_multiple_paramsets_Derecho.generate_and_submit_multi_CTSM_runs(iter_end, path_submit, path_paramset, path_CTSM_base, 
                                                                       path_archive, script_singlerun, script_clone, 
                                                                       date_start, date_end, ref_streamflow, add_flow_file,
                                                                       job_CTSMiteration, job_mode)